# What's new in Python 3.4 and below?

This show-cases new features in the form of code snippets, based on these documents:

* [What’s New In Python 3.0](https://docs.python.org/3.6/whatsnew/3.0.html)
* [What’s New In Python 3.1](https://docs.python.org/3.6/whatsnew/3.1.html)
* [What’s New In Python 3.2](https://docs.python.org/3.6/whatsnew/3.2.html)
* [What’s New In Python 3.3](https://docs.python.org/3.6/whatsnew/3.3.html)
* [What’s New In Python 3.4](https://docs.python.org/3.6/whatsnew/3.4.html)

I won't mention stuff most people know from Python 2.7 backports like print-as-a-function, integer division, unicode strings, and so on.

Python 3.0 started off with new `super()` and added `next()`. `raw_input()`, `file` type, `execfile` are all gone. `dict.has_key` removed because `in` is way more readable. Moved `reduce` to `functools.reduce` (and its use is discouraged). `reload` is now `imp.reload`. Methods accessed via their class are plain functions (no more ‘unbound methods’). Sorting a heterogeneous list no longer makes sense, due to the comparison operators being strict about only accepting compatible types. On the matter of sorting, use `key` instead of `cmp`, which is gone.

PEP-3101 replaces `%` formatting with a new approach (`.format(…)`, see [pyformat.info](https://pyformat.info/)). PEP-3107 introduces function argument and return value annotations, reflected in the `__annotations__` attribute. Keyword-only arguments are added via PEP-3102 – named parameters occurring after `*args` or bare `*` in the parameter list *must* be specified using keyword syntax in a call. Classes now pass keyword arguments to their respective metaclass. The new `nonlocal` keyword (PEP-3104) makes outer but non-global scopes writable. Finally, PEP-3132 allows extended iterable unpacking (`a, *rest, b = range(5)`).

Dict collections are now views, and the `iter` variants are no longer there. `zip`, `map` and `filter` return iterators, and `xrange` morphed into `range`.

Python 3.1 added support for `__main__.py` in ZIPs and directories, making them callable as ‘scripts’. Similarly, packages containing such a module can be started with `python -m …`. IPv6 was added to `nntplib` and `imaplib`.

Python 3.4 introduces by-default certificate verification for HTTP clients (PEP 476) and makes `pip` a commodity.

## New and Changed Syntax

In [1]:
# Dictionary comprehensions
{x: x**2 for x in range(5)}

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

In [2]:
# Set literals and set comprehensions
print({1, 2, 1}, set(), type({}), sorted({x.upper() for x in __name__}), sep='\n')

{1, 2}
set()
<class 'dict'>
['A', 'I', 'M', 'N', '_']


In [3]:
# New octal and binary literals (the old octal literals are gone)
0o720, oct(464), 0b1010, bin(10)

(464, '0o720', 10, '0b1010')

In [4]:
# Bytes literals and `bytes()`
b'abc\234', bytes('ab¢', encoding='utf-8')

(b'abc\x9c', b'ab\xc2\xa2')

The `with` statement now allows multiple context managers in a single statement (and `contextlib.nested` is deprecated).

In [5]:
import shutil
from tempfile import NamedTemporaryFile

with open('/etc/issue', 'rb') as inp, NamedTemporaryFile() as out:
    shutil.copyfileobj(inp, out)
    out.flush()
    !head -n1 {out.name}

Ubuntu 18.04.2 LTS \n \l


And `gzip.GzipFile` / `bz2.BZ2File` objects are now context managers.

Raising exceptions can now include a root cause (`raise ‹exc› from ‹cause›`), and catching is done with `except ‹exc› as ‹cause›`.

## New Library Modules

-   **asyncio**: New provisional API for asynchronous I/O
-   ensurepip: Bootstrapping the pip installer
-   **enum**: Support for enumeration types
-   importlib: A complete, portable, pure Python reference implementation of the import statement and `__import__`
-   **pathlib**: Object-oriented filesystem paths
-   selectors: High-level and efficient I/O multiplexing, built upon the select module primitives
-   **statistics**: A basic numerically stable statistics library
-   tracemalloc: Trace Python memory allocations

## Changes to Built-ins

### Ordered Dictionaries (v3.1, PEP 372)

The new `collections.OrderedDict` class allows you to retain insertion order of items. Note that Python 3.7 makes this a language feature for `dict`, and 3.6 already implements it *incidentally*.

In [6]:
from collections import OrderedDict
o = OrderedDict(z=1, x=2, a=3)
o, dict(o)

(OrderedDict([('z', 1), ('x', 2), ('a', 3)]), {'z': 1, 'x': 2, 'a': 3})

If a new entry overwrites an existing entry, the original insertion position is left unchanged.

In [7]:
o['x'] = -2
o

OrderedDict([('z', 1), ('x', -2), ('a', 3)])

Deleting an entry and reinserting it will move it to the end.

In [8]:
del o['x']; o['x'] = 4
o

OrderedDict([('z', 1), ('a', 3), ('x', 4)])

The standard library takes advantage of ordering at several places.

### Format Specifier for Thousands Separator (v3.1, PEP 378)

The built-in `format` function and `str.format` offer a simple, non-locale aware way to format a number with a thousands separator. Locale-aware applications should use the existing `n` format specifier which already has some support for thousands separators.

In [9]:
format(1234567, ',d')

'1,234,567'

In [10]:
format(1234567.89, ',.2f')

'1,234,567.89'

In [11]:
format(12345.6 + 8901234.12j, ',f')

'12,345.600000+8,901,234.120000j'

In [12]:
import decimal
format(decimal.Decimal('1234567.89'), ',f')

'1,234,567.89'

The supported types are `int`, `float`, `complex`, and `decimal.Decimal`.

### Small Changes & Additions

In [13]:
# Unnumbered fields in format strings
'{} {}'.format('one', 'two')

'one two'

In [14]:
# round(x, n) keeps the type of passed integers,
# instead of converting to float as before.
i = round(1156, -2)
i, type(i)

(1200, int)

In [15]:
# Bit-length of ints
(42).bit_length(), bin(42), (65535).bit_length()

(6, '0b101010', 16)

In [16]:
# Use this instead of `callable(f)`
from collections import Callable
isinstance(open, Callable), isinstance(None, Callable)

(True, False)

In [17]:
# Convenient counting of unique items in a sequence or iterable
from collections import Counter
Counter(['red', 'blue', 'red', 'green', 'blue', 'blue'])

Counter({'red': 2, 'blue': 3, 'green': 1})

In [18]:
# combinations_with_replacement
import itertools
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [19]:
# compress mimics its namesake from APL
list(itertools.compress(data=range(5), selectors=[0,0,1,1,0]))

[2, 3]

In [20]:
# count takes a step argument
c = itertools.count(start=1.1, step=0.4)
[next(c), next(c), next(c), next(c)]

[1.1, 1.5, 1.9, 2.3]

In [21]:
# namedtuple supports invalid fieldname as `_N` on demand
from collections import namedtuple
nt = namedtuple('nt', 'ok pric€ last', rename=True)
nt.__doc__

'nt(ok, _1, last)'

In [22]:
# sys.version_info is a named tuple
import sys
sys.version_info

sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)

In [23]:
# Constants for the seek method
import io
io.SEEK_SET, io.SEEK_CUR, io.SEEK_END

(0, 1, 2)

Use shortened float output representation, if possible – without manipulating the underlying values, see below addition. See [0.30000000000000004.com](http://0.30000000000000004.com/) for some background.

In [24]:
1.1, 2.2, 3.3, 1.1 + 2.2

(1.1, 2.2, 3.3, 3.3000000000000003)

In [25]:
# Create decimal from from binary float, with exact representation
from decimal import Decimal
Decimal.from_float(1.1), Decimal(str(1.1))

(Decimal('1.100000000000000088817841970012523233890533447265625'),
 Decimal('1.1'))

The `maketrans()` and `translate()` methods are now uniformly available for `str`, `bytes`, and `bytearray`.

In [26]:
updn = str.maketrans('aeiou', 'ɐǝıon')
updn, str.maketrans(dict(a=''))

({97: 592, 101: 477, 105: 305, 111: 111, 117: 110}, {97: ''})

In [27]:
'Lorem ipsum'.translate(updn)

'Lorǝm ıpsnm'

`help()` now supports symbols, like `~`.

In [28]:
import io, re
from unittest.mock import patch
with patch('sys.stdout', new=io.StringIO()) as out:
    help('~')
re.sub(' +', ' ', [x for x in out.getvalue().splitlines() if '~' in x][0])

'| "+x", "-x", "~x" | Positive, negative, bitwise NOT |'